# Table of Contents

## a) Dataset selection and description

## b) Implement sigmoid and ReLU layers

## c) Implement softmax layer

## d) Implement dropout

## e) Implement a fully parametrizable neural network class

## f) Implement optimizer

## g) Evaluate different neural network architectures/parameters, present and discuss your results. 

## h) Code quality and report presentation 